In [ ]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
import numpy as np
import random
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

def apply_noise_to_labels(labels, noise_level, noise_type='symmetric'):
    noisy_labels = labels.copy()
    n_classes = 10  # CIFAR-10 has 10 classes
    label_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}  # Asymmetric mapping

    for i in range(len(noisy_labels)):
        if random.random() < noise_level:
            if noise_type == 'symmetric':
                # Create a list of possible labels excluding the original one
                possible_labels = list(range(n_classes))
                possible_labels.remove(noisy_labels[i])
                noisy_labels[i] = random.choice(possible_labels)
            elif noise_type == 'asymmetric':
                # Only apply noise to labels that are in the asymmetric mapping
                if noisy_labels[i] in label_mapping:
                    noisy_labels[i] = label_mapping[noisy_labels[i]]
    return to_categorical(noisy_labels, num_classes=n_classes)


def verify_noise(labels, noisy_labels, noise_level):
    total_labels = len(labels)
    num_changed = np.sum(np.argmax(labels, axis=1) != np.argmax(noisy_labels, axis=1))
    changed_percentage = num_changed / total_labels
    print(f"Expected noise level: {noise_level}, Actual changed labels: {changed_percentage:.2f}")
    return abs(changed_percentage - noise_level) < 0.05  # Allowing a small deviation

# Example usage
trainX, trainY, testX, testY = load_dataset()
noisy_trainY = apply_noise_to_labels(np.argmax(trainY, axis=1), 0.3, 'symmetric')
is_noise_correct = verify_noise(trainY, noisy_trainY, 0.3)
print("Noise application is correct:", is_noise_correct)
noisy_trainY = apply_noise_to_labels(np.argmax(trainY, axis=1), 0, 'symmetric')
is_noise_correct = verify_noise(trainY, noisy_trainY, 0)
print("Noise application is correct:", is_noise_correct)
def verify_asymmetric_noise(labels, noisy_labels, noise_level, label_mapping):
    # 计算所有标签的总数
    total_labels = len(labels)

    # 计算实际改变的标签数量
    actual_changes = sum((labels == original) & (noisy_labels != original)
                         for original in label_mapping)
    total_actual_changes = actual_changes.sum()

    # 计算改变标签的比例（基于所有标签的总数）
    change_percentage = total_actual_changes / total_labels
    print(f"Expected asymmetric noise level: {noise_level}, Actual changed labels: {change_percentage:.2f}")
    return abs(change_percentage - noise_level) < 0.05  # 允许一定的偏差

# 示例使用
trainX, trainY, testX, testY = load_dataset()
label_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}
noisy_trainY = apply_noise_to_labels(np.argmax(trainY, axis=1), 0.3, 'asymmetric')
is_asymmetric_noise_correct = verify_asymmetric_noise(np.argmax(trainY, axis=1), np.argmax(noisy_trainY, axis=1), 0.15, label_mapping)
print("Asymmetric noise application is correct:", is_asymmetric_noise_correct)

# baseline model with dropout on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.2))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	pyplot.show()
	pyplot.close()

import numpy as np
import random
from keras.utils import to_categorical

def apply_noise_to_labels(labels, noise_level, noise_type='symmetric'):
    noisy_labels = labels.copy()
    n_classes = 10  # CIFAR-10 has 10 classes
    label_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}  # Asymmetric mapping

    for i in range(len(noisy_labels)):
        if random.random() < noise_level:
            if noise_type == 'symmetric':
                # Create a list of possible labels excluding the original one
                possible_labels = list(range(n_classes))
                possible_labels.remove(noisy_labels[i])
                noisy_labels[i] = random.choice(possible_labels)
            elif noise_type == 'asymmetric':
                # Only apply noise to labels that are in the asymmetric mapping
                if noisy_labels[i] in label_mapping:
                    noisy_labels[i] = label_mapping[noisy_labels[i]]
    return to_categorical(noisy_labels, num_classes=n_classes)

def run_test_harness(noise_level=0.0, noise_type='symmetric'):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()

    # apply noise to labels
    trainY_noisy = apply_noise_to_labels(np.argmax(trainY, axis=1), noise_level, noise_type)

    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)

    # define model
    model = define_model()

    # fit model
    history = model.fit(trainX, trainY_noisy, epochs=22, batch_size=64, validation_data=(testX, testY), verbose=2)

    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=2)
    print(f'Noise Level {noise_level} - Noise Type {noise_type} - Accuracy: {acc * 100.0}%')

    # learning curves
#     summarize_diagnostics(history)

# print ("\n\n\n")
# print(f"#######   Training with 0 noise level: ########")
# run_test_harness(0, 'symmetric')
import torch
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

noise_types = ['symmetric']
seeds = range(10)  # seeds repo

for seed in seeds:
    torch.manual_seed(seed)  # 设置随机种子
    np.random.seed(seed)  # 对于numpy操作也设置随机种子
    random.seed(seed)  # 对于Python内置的random库也设置随机种子

    for noise_type in noise_types:
        for noise_level in noise_levels:
            print(f"####### Training with Seed: {seed} type: {noise_type} noise level: {noise_level} ########")
            run_test_harness(noise_level, noise_type)
            print("\n\n\n")

Expected noise level: 0.3, Actual changed labels: 0.30
Noise application is correct: True
Expected noise level: 0, Actual changed labels: 0.00
Noise application is correct: True
Expected asymmetric noise level: 0.15, Actual changed labels: 0.15
Asymmetric noise application is correct: True
####### Training with Seed: 0 type: symmetric noise level: 0.1 ########


Epoch 1/22
